In [1]:
import pandas as pd

## 1. Data collection

In [2]:
Data=pd.read_csv("insurance_pre.csv")
Data

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
Data_set=pd.get_dummies(Data, drop_first=True)
Data_set

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [4]:
Data_set.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

## 2. Input and Output split

In [5]:
Independent=Data_set[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
Independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [6]:
Dependent=Data_set[['charges']]
Dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


## 3.Split train and test data

In [7]:
#Here we are using Gird Search CV (cross validation) so no need to use train and test split, Gird will do that all the method.

## 4. Model creation

In [8]:
!pip install lightgbm

In [10]:
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor

p_grid={'n_estimators':[10,50,100], 'objective':['regression','poisson', 'mape'], 'boosting_type':['gbdt','dart', 'rf']}

#In default CV take Kfold method to do cross validate we can mention the cv in code like below line
KFold=KFold(n_splits=5,shuffle=True, random_state=42)

grid=GridSearchCV(LGBMRegressor(random_state=42), p_grid, cv=KFold, refit=True, verbose=3, n_jobs=-1)
grid.fit(Independent, Dependent)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000165 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 1338, number of used features: 5
[LightGBM] [Info] Start training from score 9.493293


C:\Users\smani\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
45 fits failed out of a total of 135.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\smani\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\smani\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 1398, in fit
    super().fit(
  File "C:\Users\smani\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 1049, in fit
    self._Booster = train(
  File "C:\Users\smani\anaconda3\lib\site-packages\lightgbm\engine.py", line 297, i

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=LGBMRegressor(random_state=42), n_jobs=-1,
             param_grid={'boosting_type': ['gbdt', 'dart', 'rf'],
                         'n_estimators': [10, 50, 100],
                         'objective': ['regression', 'poisson', 'mape']},
             verbose=3)

## 5. Evalution metric

In [11]:
#since we are using GridSearchCV so it will do Evalution the metric and give the best model

In [12]:
# Store the result in re variable
re=grid.cv_results_
print("The Score value for the best model {}".format(grid.best_params_),"Score :",grid.best_score_)

The Score value for the best model {'boosting_type': 'gbdt', 'n_estimators': 50, 'objective': 'poisson'} Score : 0.8448487692452158


In [13]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_boosting_type,param_n_estimators,param_objective,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.059215,0.010782,0.005621,0.000802,gbdt,10,regression,"{'boosting_type': 'gbdt', 'n_estimators': 10, ...",7.708449e-01,7.699729e-01,7.776558e-01,7.000346e-01,7.250019e-01,7.487020e-01,0.030696,11
1,0.059893,0.008556,0.005624,0.000803,gbdt,10,poisson,"{'boosting_type': 'gbdt', 'n_estimators': 10, ...",5.676759e-01,5.760122e-01,5.730630e-01,5.102824e-01,5.281825e-01,5.510432e-01,0.026717,14
2,0.061626,0.005907,0.004366,0.000701,gbdt,10,mape,"{'boosting_type': 'gbdt', 'n_estimators': 10, ...",6.473927e-01,6.041396e-01,5.903206e-01,5.294576e-01,6.099500e-01,5.962521e-01,0.038372,13
3,0.231295,0.030106,0.005469,0.000733,gbdt,50,regression,"{'boosting_type': 'gbdt', 'n_estimators': 50, ...",8.725555e-01,8.428396e-01,8.727317e-01,7.972607e-01,8.341627e-01,8.439100e-01,0.028013,3
4,0.213132,0.024753,0.005068,0.000971,gbdt,50,poisson,"{'boosting_type': 'gbdt', 'n_estimators': 50, ...",8.712959e-01,8.492411e-01,8.794349e-01,7.929742e-01,8.312977e-01,8.448488e-01,0.030950,1
5,0.207160,0.015206,0.005466,0.000731,gbdt,50,mape,"{'boosting_type': 'gbdt', 'n_estimators': 50, ...",8.427146e-01,7.829405e-01,8.069973e-01,7.538060e-01,8.110491e-01,7.995015e-01,0.029730,9
6,0.429300,0.009751,0.005921,0.000126,gbdt,100,regression,"{'boosting_type': 'gbdt', 'n_estimators': 100,...",8.676204e-01,8.338896e-01,8.593166e-01,7.857975e-01,8.229539e-01,8.339156e-01,0.029040,4
7,0.427269,0.009108,0.005973,0.001274,gbdt,100,poisson,"{'boosting_type': 'gbdt', 'n_estimators': 100,...",8.740265e-01,8.419867e-01,8.768896e-01,7.909996e-01,8.358223e-01,8.439449e-01,0.031187,2
8,0.439639,0.003721,0.005239,0.000962,gbdt,100,mape,"{'boosting_type': 'gbdt', 'n_estimators': 100,...",8.538373e-01,7.940176e-01,8.284338e-01,7.827853e-01,8.221317e-01,8.162411e-01,0.025338,6
9,0.042543,0.002429,0.004014,0.000002,dart,10,regression,"{'boosting_type': 'dart', 'n_estimators': 10, ...",6.943515e-01,6.981256e-01,6.996148e-01,6.280041e-01,6.496140e-01,6.739420e-01,0.029539,12


In [22]:
age=int(input("Enter your age :"))
bmi=float(input("Enter your BMI : "))
children=int(input("Enter Children count:"))
Gen=int(input("Enter the gander if male enter 1, if female enter 0 :"))
smok=int(input("Enter if somker enter 1, if no somker enter 0 :"))

Enter your age :22
Enter your BMI : 28.900
Enter Children count:2
Enter the gander if male enter 1, if female enter 0 :0
Enter if somker enter 1, if no somker enter 0 :1


In [23]:
result=grid.predict([[age,bmi,children,Gen,smok]])

In [24]:
result

array([19160.17536507])